In [1]:
import pandas as pd
import numpy as np

In [2]:
data = pd.read_csv("cme_events.csv")
data.head()

,Sno,src_loc,date,time,ang_wdt,vcdaw,spce_spd,fht_spd2,20RS_spd2,accel,...,meanB,mc,v_trans,trans_time,icme_sz,Icme_min_Bz,sheath_spd,sheath_sz,sheath_dst,sheath Bz
0,1,NaN,1996/12/19,16:30,130.0,469.0,NaN,NaN,NaN,NaN,...,10.0,2,435,95,NaN,NaN,NaN,NaN,NaN,NaN
1,2,S18E06,1997/01/06,15:10,360.0,136.0,232.0,224.0,319.0,4.1,...,14.0,2,507,81,NaN,NaN,NaN,NaN,NaN,NaN
2,3,S38W31,1997/02/07,0:30,360.0,490.0,584.0,718.0,635.0,14.3,...,8.0,2,683,60,NaN,NaN,NaN,NaN,NaN,NaN
3,4,S28E19,1997/04/07,14:27,360.0,878.0,1115.0,905.0,896.0,3.3,...,20.0,2,552,74,NaN,NaN,NaN,NaN,NaN,NaN
4,5,N21W19,1997/05/12,5:30,360.0,464.0,749.0,335.0,220.0,-15.0*1,...,21.0,2,616,67,NaN,NaN,NaN,NaN,NaN,NaN


In [3]:
drop_col=['Sno']
data.drop(drop_col, inplace=True, axis=1)
data.head()

,src_loc,date,time,ang_wdt,vcdaw,spce_spd,fht_spd2,20RS_spd2,accel,mass,...,meanB,mc,v_trans,trans_time,icme_sz,Icme_min_Bz,sheath_spd,sheath_sz,sheath_dst,sheath Bz
0,NaN,1996/12/19,16:30,130.0,469.0,NaN,NaN,NaN,NaN,5.1e+14*2,...,10.0,2,435,95,NaN,NaN,NaN,NaN,NaN,NaN
1,S18E06,1997/01/06,15:10,360.0,136.0,232.0,224.0,319.0,4.1,5.8e+14*2,...,14.0,2,507,81,NaN,NaN,NaN,NaN,NaN,NaN
2,S38W31,1997/02/07,0:30,360.0,490.0,584.0,718.0,635.0,14.3,4.1e+15*2,...,8.0,2,683,60,NaN,NaN,NaN,NaN,NaN,NaN
3,S28E19,1997/04/07,14:27,360.0,878.0,1115.0,905.0,896.0,3.3,1.0e+16*2,...,20.0,2,552,74,NaN,NaN,NaN,NaN,NaN,NaN
4,N21W19,1997/05/12,5:30,360.0,464.0,749.0,335.0,220.0,-15.0*1,4.2e+15*2,...,21.0,2,616,67,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
def datainfo():
    print(data.shape)
    print(data.columns)
    print(data.dtypes)

In [16]:
datainfo()

(287, 37)
Index(['src_loc', 'datetime', 'ang_wdt', 'vcdaw', 'spce_spd', 'fht_spd2',
       '20RS_spd2', 'accel', 'mass', 'KE', 'mpa', 'src_typ', 'NOAA',
       'onset_time', 'hale_class', 'flr_class', 'flr_dttm', 'icme_dst_dttm',
       'plsm_strt_dttm', 'plsm_end_dttm', 'comp_strt', 'comp_end', 'qual',
       'dv', 'v', 'min_dst_icme', 'v_max', 'meanB', 'mc', 'v_trans',
       'trans_time', 'icme_sz', 'Icme_min_Bz', 'sheath_spd', 'sheath_sz',
       'sheath_dst', 'sheath Bz'],
      dtype='object')
src_loc                   object
datetime          datetime64[ns]
ang_wdt                  float64
vcdaw                    float64
spce_spd                 float64
fht_spd2                 float64
20RS_spd2                float64
accel                     object
mass                      object
KE                        object
mpa                      float64
src_typ                   object
NOAA                     float64
onset_time                object
hale_class                object


In [7]:
data.describe()

,ang_wdt,vcdaw,spce_spd,fht_spd2,20RS_spd2,mpa,NOAA,v,v_max,meanB,Icme_min_Bz,sheath_spd,sheath_sz,sheath_dst,sheath Bz
count,286.000000,286.000000,182.000000,283.000000,283.000000,285.000000,267.000000,287.000000,273.000000,273.000000,75.000000,73.00000,73.000000,67.000000,69.000000
mean,297.835664,788.423077,1202.120879,787.190813,800.579505,187.708772,10835.651685,492.423031,571.062271,12.765568,-11.378667,566.60274,28.904110,-60.791045,-8.266667
std,111.711742,497.226976,547.326689,465.892888,509.683677,102.716464,1584.881669,158.422619,179.267217,30.030070,9.019413,157.64832,18.683151,59.499819,6.700893
min,10.000000,104.000000,129.000000,119.000000,0.000000,0.000000,8009.000000,310.000000,310.000000,1.000000,-45.900000,338.00000,1.000000,-383.000000,-32.300000
25%,211.250000,441.500000,838.500000,457.500000,424.500000,90.000000,9408.500000,400.000000,460.000000,7.000000,-15.250000,464.00000,15.000000,-76.500000,-10.200000
50%,360.000000,673.000000,1108.500000,665.000000,682.000000,189.000000,10930.000000,460.000000,530.000000,10.000000,-8.900000,541.00000,24.000000,-41.000000,-6.600000
75%,360.000000,1061.000000,1415.750000,1038.000000,1085.000000,275.000000,12058.000000,525.000000,650.000000,14.000000,-5.450000,634.00000,40.000000,-23.000000,-4.100000
max,1233.000000,3387.000000,3656.000000,3142.000000,3330.000000,358.000000,13514.000000,1631.000000,1900.000000,500.000000,-0.100000,1161.00000,80.000000,-1.000000,-0.300000


In [18]:
data.isna().sum()

src_loc            13
datetime            0
ang_wdt             1
vcdaw               1
spce_spd          105
fht_spd2            4
20RS_spd2           4
accel               4
mass               21
KE                 21
mpa                 2
src_typ             1
NOAA               20
onset_time         15
hale_class         20
flr_class         139
flr_dttm          223
icme_dst_dttm      25
plsm_strt_dttm      1
plsm_end_dttm      36
comp_strt         150
comp_end          151
qual                2
dv                 14
v                   3
min_dst_icme        0
v_max              17
meanB              17
mc                 17
v_trans             5
trans_time          0
icme_sz           208
Icme_min_Bz       212
sheath_spd        214
sheath_sz         214
sheath_dst        220
sheath Bz         218
dtype: int64

In [9]:
data.insert(1,'datetime',pd.to_datetime(data['date'] + ' ' + data['time'], errors='coerce')) # Join date time column 
data.dropna(subset=['datetime'], inplace = True) #drop rows where datetime is NaT
data.drop(columns=['date','time'], inplace = True)

In [10]:
numeric_col = ['ang_wdt','fht_spd2','20RS_spd2']
data[numeric_col] = data[numeric_col].apply(pd.to_numeric)

In [11]:
clmns = ['flr_dt','flr_tm','icme_dst_dt','icme_dst_tm','plsm_strt_dt','plsm_strt_tm','plsm_end_dt','plsm_end_tm']
pos = [data.columns.get_loc(col) for col in clmns]
print(pos)

[16, 17, 18, 19, 20, 21, 22, 23]


In [12]:
def datetimeconvert(pos,colname,dtcol,tmcol):
    data.insert(pos,colname,pd.to_datetime(data[dtcol] + ' ' + data[tmcol], errors ='coerce'))
    data.drop(columns=[dtcol,tmcol], inplace = True)

In [13]:
datetimeconvert(16,'flr_dttm','flr_dt','flr_tm') 
datetimeconvert(17,'icme_dst_dttm','icme_dst_dt','icme_dst_tm')       
datetimeconvert(18,'plsm_strt_dttm','plsm_strt_dt','plsm_strt_tm')
datetimeconvert(19,'plsm_end_dttm','plsm_end_dt','plsm_end_tm')

C:\Users\ayaan\AppData\Local\Temp\ipykernel_21024\162378449.py:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  data.insert(pos,colname,pd.to_datetime(data[dtcol] + ' ' + data[tmcol], errors ='coerce'))


In [17]:
datainfo()

(287, 37)
Index(['src_loc', 'datetime', 'ang_wdt', 'vcdaw', 'spce_spd', 'fht_spd2',
       '20RS_spd2', 'accel', 'mass', 'KE', 'mpa', 'src_typ', 'NOAA',
       'onset_time', 'hale_class', 'flr_class', 'flr_dttm', 'icme_dst_dttm',
       'plsm_strt_dttm', 'plsm_end_dttm', 'comp_strt', 'comp_end', 'qual',
       'dv', 'v', 'min_dst_icme', 'v_max', 'meanB', 'mc', 'v_trans',
       'trans_time', 'icme_sz', 'Icme_min_Bz', 'sheath_spd', 'sheath_sz',
       'sheath_dst', 'sheath Bz'],
      dtype='object')
src_loc                   object
datetime          datetime64[ns]
ang_wdt                  float64
vcdaw                    float64
spce_spd                 float64
fht_spd2                 float64
20RS_spd2                float64
accel                     object
mass                      object
KE                        object
mpa                      float64
src_typ                   object
NOAA                     float64
onset_time                object
hale_class                object
